In [4]:
import pandas as pd

hate_stA_train = pd.read_csv(".\OnlyText\stA\hate_stA_train.csv")
nohate_stA_train = pd.read_csv(".\OnlyText\stA\put.csv")
stA_eval = pd.read_csv(".\OnlyText\stA\stA_eval.csv")
stA_test = pd.read_csv(".\OnlyText\stA\stA_test.csv")
eval_subtaskA = pd.read_csv(".\OnlyText\stA\eval_subtaskA.csv")


import re
import spacy
nlp = spacy.load("en_core_web_sm")

'''
def preprocess_synopsis(text):
    text.replace("\n"," ")
    text.replace("\r","")
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower()).replace('\n', '').replace('\r', '')
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text
'''


problematic_indices_eval = [
    39391,
48843,
94515,
81181,
69034,
47368,
29455,
56174,
57069,
74368,
67470,
73193,
50784,
66268,
84383,
98442,
99385
]

def labeler(sample_id, df):
    for column, row in df.iterrows():
        if str(row['index']) == str(sample_id):
            a = int(row['Label'])
            if a == 1:
                return 1 #'hate speech'
            else:
                return 0 #'no hate speech'
    print("Function failed")

training_data = pd.DataFrame(columns=['Sample', 'Text', 'Label'])
hate = 1# 'hate speech' #1
no_hate = 0#'no hate speech' #0
count = 1
for column, row in stA_eval.iterrows():
    sample_id = row['filename'].replace("/content/drive/MyDrive/CASE2023_Task4/CASE2023_TASK4_EvalData/subtaskA/","").replace(".jpg","")
    sample_id_new = sample_id
    if int(sample_id) in problematic_indices_eval:
        sample_id_new = sample_id+"(1)"
    sample_text = row['text']
    sample_text = preprocess_synopsis(sample_text)
    training_data.loc[count] = [sample_id_new, sample_text, labeler(sample_id, eval_subtaskA)]
    count+=1
for column, row in hate_stA_train.iterrows():
    sample_id = row['filename'].replace("/content/drive/MyDrive/CASE2023_Task4/CASE2023_TASK4_TrainData/subTaskA/Hate Speech/","").replace(".jpg","")
    sample_text = row['text']
    sample_text = preprocess_synopsis(sample_text)
    training_data.loc[count] = [sample_id, sample_text, hate]
    count+=1
for column, row in nohate_stA_train.iterrows():
    sample_id = row['filename'].replace("/content/drive/MyDrive/CASE2023_Task4/CASE2023_TASK4_TrainData/subTaskA/No Hate Speech/","").replace(".jpg","")
    sample_text = row['text']
    sample_text = preprocess_synopsis(sample_text)
    training_data.loc[count] = [sample_id, sample_text, no_hate]
    count+=1

print(training_data)

     Sample                                               Text  Label
1     10087  truth texas   republican president donald trum...      1
2     10113  putin stressing tf hearing spilled bunch paint...      1
3     10203  ip interference ukraine lead consequences ve s...      0
4     10256    carolina forward thread forwardcarolina sena...      0
5     10270             s fuck putin means everybody wants sex      1
...     ...                                                ...    ...
4039  99632  protestors world rally support ukraine storyfulap      0
4040  99658  rt live q   mar     home   russia   fsu wwiii ...      0
4041  99773  ragingeve costa mesa couple barely escape ukra...      0
4042  99882  sign contribute guardian   years news opinion ...      0
4043  99927  rob lee ralee colonel alexey gorobets commande...      0

[4043 rows x 3 columns]


In [8]:
'''
for column, row in training_data.iterrows():
    if row['Label'] == 0:
        print(row['Text'])
        break

'Ip Interference in Ukraine move would lead to consequences you've never seen: Putin As he announced a military operation in Ukraine, Russian President Vladimir Putin warned other countries that any attempt to interfere with the Russian action would lead to "consequences you have never seen". He said the attack was needed to protect civilians in eastern Ukraine, a claim that the US had predicted he would falsely make to justify an invasion. short by Ankush Verma / 24 Feb, 2022 GET IT ON Google Play Download on the App Store inshorts'
'''

test_datas = pd.DataFrame(columns=['Sample', 'Text'])
count = 1
for column, row in stA_test.iterrows():
    sample_id = row['filename'].replace("/content/drive/MyDrive/CASE2023_Task4/CASE2023_TASK4_TestData/subtaskA/","").replace(".jpg","")
    sample_text = row['text']
    test_datas.loc[count] = [sample_id, sample_text]
    count+=1
    
print(test_datas)

    Sample                                               Text
1    10113    When you keep all the good rations for yourself
2    10165  Media 9:28 1 Tucker Carlson, downplaying Russi...
3    10287  Andreeva Bay nuclear waste storage Bolshaya Lo...
4    10443  So many electricians in the Ukraine but no ele...
5    10532  Daily Mail MORE STORIES Q Russia has fired 'ab...
..     ...                                                ...
439  99353   UKRAINIAN RESISTANCE TO RUSSIA imgflip.com A30 $
440  99743  Patricia Arquette @PattyArquette Well for Lord...
441  99744  4:19 1 Amazon's response to the situation in U...
442  99764  Fox News just asked Trump what he'd do differe...
443  99828  ALL I SAID WAS... NO CHANCE OF WAR IN UKRAINE ...

[443 rows x 2 columns]


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score
import torch

# Step 1: Prepare the environment
df = training_data

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Step 2: Tokenize the text
def tokenize_text(text):
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt'
    )
    return encoded['input_ids'], encoded['attention_mask']

train_tokens = train_df['Text'].apply(tokenize_text)
test_tokens = test_df['Text'].apply(tokenize_text)

# Step 3: Prepare the data for BERT
train_inputs = torch.cat([token[0] for token in train_tokens])
train_masks = torch.cat([token[1] for token in train_tokens])
train_labels = torch.tensor(train_df['Label'].values)

test_inputs = torch.cat([token[0] for token in test_tokens])
test_masks = torch.cat([token[1] for token in test_tokens])
test_labels = torch.tensor(test_df['Label'].values)

# Step 4: Fine-tune BERT for classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

optimizer = AdamW(model.parameters(), lr=2e-5)

batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_loader = DataLoader(train_data, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_loader = DataLoader(test_data, batch_size=batch_size)

epochs = 4

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]

        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)

    # Evaluation on the test set
    model.eval()
    total_correct = 0
    true_labels = []
    predicted_labels = []

    for batch in test_loader:
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        _, predicted = torch.max(logits, dim=1)
        total_correct += (predicted == labels).sum().item()

        true_labels.extend(labels.numpy())
        predicted_labels.extend(predicted.numpy())

    accuracy = total_correct / len(test_df)
    f1 = f1_score(true_labels, predicted_labels)

    print(f'Epoch {epoch + 1}: Train Loss = {avg_train_loss}, Test Accuracy = {accuracy}, F1 Score = {f1}')



c:\Users\CHQ\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint o

Epoch 1: Train Loss = 0.5680123288643184, Test Accuracy = 0.7812113720642769, F1 Score = 0.7844092570036539
Epoch 2: Train Loss = 0.40881286293561825, Test Accuracy = 0.7725587144622992, F1 Score = 0.7745098039215685
Epoch 3: Train Loss = 0.26170835863303377, Test Accuracy = 0.8034610630407911, F1 Score = 0.8199320498301246
Epoch 4: Train Loss = 0.1940963120667695, Test Accuracy = 0.8220024721878862, F1 Score = 0.8444924406047516


In [9]:
def predict_text(text):
    inputs = tokenize_text(text)
    input_ids = inputs[0]
    attention_mask = inputs[1]

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]

    _, predicted_label = torch.max(logits, dim=1)
    return predicted_label.item()

predicciones_chidas = []
for column, row in test_datas.iterrows():
    sample = row['Sample']
    texto = row['Text']
    predicted_label = predict_text(texto)
    linea = '{"index": '+ str(sample)+ ', "prediction": '+ str(predicted_label)+ '}'
    predicciones_chidas.append(linea)

In [12]:
import json

jason = open('BERT_prepros_submission.json', 'w')
for i in range(len(predicciones_chidas)-1):
    jason.write(predicciones_chidas[i])
    jason.write('\n')
jason.write(predicciones_chidas[-1])
jason.close()